## Introduction

In this tutorial, we’ll walk through the process of interacting with a Google Cloud Storage (GCS) bucket named dauphine-bucket, specifically focusing on the data directory within the bucket. We’ll cover how to:

- List all files in the bucket’s data directory.
- Retrieve information about a specific file.
- Read files using the Unstructured library.
- Visualize the extracted documents with LangChain.

This guide is intended for users who are familiar with Python and basic cloud storage concepts.

Prerequisites

Before we begin, ensure you have the following:

- Python 3.x installed on your system.
- Access to the GCP bucket dauphine-bucket/data with the necessary permissions.
- Google Cloud SDK installed and authenticated. You can authenticate by running:

In [15]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=5eoApHSMjHpXZ0qXGbYK6gtZw9iuGq&access_type=offline&code_challenge=LU9XTFdd5OJuYGUq-_U-Q6RkH3bciVnQ--6pG961YsI&code_challenge_method=S256


You are now logged in as [abdelhedi.youssef8@gmail.com].
Your current project is [dauphine-437611].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


The following Python libraries installed:
- google-cloud-storage
- unstructured
- langchain

To know more about the libraries, you can visit the following links:
- [google-cloud-storage](https://googleapis.dev/python/storage/latest/index.html)
- [unstructured](https://docs.unstructured.io/examplecode/codesamples/oss/vector-database)
- [langchain](https://langchain.readthedocs.io/en/latest/)


In [13]:
%pip install -q google-cloud-storage unstructured langchain python-magic sqlalchemy langchain_google_cloud_sql_pg
%pip install -q "unstructured[pptx]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# I. Listing and loading files from a GCS bucket

### I.1. Listing Files in the GCP Bucket

Explanation:

To interact with a GCS bucket, we’ll use the google-cloud-storage library. We’ll initialize a client, access the bucket, and list all the files within the data directory.

Code:

In [16]:
# Import the necessary library
from google.cloud import storage

# Initialize a client
client = storage.Client()

# Access the bucket
bucket_name = 'dauphine-bucket'
bucket = client.get_bucket(bucket_name)

# List all files in the 'data' directory
blobs = bucket.list_blobs(prefix='data/')

print("Files in 'dauphine-bucket/data':")
for blob in blobs:
    print(blob.name)

Files in 'dauphine-bucket/data':
data/
data/1 - Gen AI - Dauphine Tunis.pptx
data/2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx
data/2.2  - Transformers - Gen AI - Dauphine Tunis.pptx
data/3 - Retrieval Augmented Generation - Gen AI - Dauphine Tunis.pptx


Output Explanation:

Running this code will display all the file paths within the data directory of the bucket. The prefix='data/' parameter ensures we only get files from that specific directory.

### I.2. Getting Information About One File


Explanation:

Sometimes, you may need detailed information about a specific file, such as its size, content type, or the last time it was updated. We’ll retrieve this metadata for a chosen file.


In [17]:
# Specify the file path (replace with an actual file from your bucket)
file_path = 'data/1 - Gen AI - Dauphine Tunis.pptx'

# Get the blob object
blob = bucket.get_blob(file_path)

# TODO
if blob:
    print(f"Information for '{file_path}':")
    print(f"Size: {blob.size} bytes")
    print(f"Content Type: {blob.content_type}")
    print(f"Updated On: {blob.updated}") 
    print(f"Blob name: {blob.name}")
else:
    print(f"File '{file_path}' not found in the bucket.")


Information for 'data/1 - Gen AI - Dauphine Tunis.pptx':
Size: 6724048 bytes
Content Type: application/vnd.openxmlformats-officedocument.presentationml.presentation
Updated On: 2024-10-07 09:52:30.256000+00:00
Blob name: data/1 - Gen AI - Dauphine Tunis.pptx


Output Explanation:

This code will output metadata about the specified file. Make sure to replace 'data/your_file.ext' with the actual file path.

### I.3. Reading Files with Unstructured

Explanation:

The Unstructured library allows us to parse and process unstructured data from various file formats. We’ll download a file from the bucket and use Unstructured to read and extract its content.

In [ ]:
# Import necessary libraries
from turtle import mode
from langchain_core.documents import Document
import os
from google.cloud.storage.bucket import Bucket
from langchain_unstructured import UnstructuredLoader



DOWNLOADED_LOCAL_FIRECTORY = "./downloaded_files"

# Function to download the file: file_path from the GCS Bucket
def download_file_from_bucket(bucket: Bucket, file_path: str) -> str:
    # Download the file locally
    blob = bucket.blob(file_path)

    # data/1 - Gen AI - Dauphine Tunis.pptx' -> 1 - Gen AI - Dauphine Tunis.pptx'
    local_file_name = os.path.basename(file_path)

    # ./downloaded_files/1 - Gen AI - Dauphine Tunis.pptx
    local_filepath = os.path.join(DOWNLOADED_LOCAL_FIRECTORY, local_file_name)

    blob.download_to_filename(local_filepath)
    print(f"Downloaded '{file_path}' to '{local_file_name}'")
    return local_filepath


def read_file_from_local(local_filepath: str) -> list[Document]:
    # Use Unstructured to read the file
    loader = UnstructuredLoader(file_path=local_filepath) # TODO
    documents = loader.load() # TODO
    return documents


In [ ]:
# Load all the
blobs = list(bucket.list_blobs(prefix='data/'))
documents: list[Document] = []
if blobs:
    for blob in blobs:
        try:
            local_filepath = download_file_from_bucket(bucket, blob.name)
            documents.extend(read_file_from_local(local_filepath))
        except Exception as e:
            print(f"An error occurred while processing '{blob.name}': {e}")
else:
    print("No files found in the 'data' directory.")

An error occurred while processing 'data/': [Errno 2] No such file or directory: './downloaded_files\\'
An error occurred while processing 'data/1 - Gen AI - Dauphine Tunis.pptx': 403 GET https://storage.googleapis.com/download/storage/v1/b/dauphine-bucket/o/data%2F1%20-%20Gen%20AI%20-%20Dauphine%20Tunis.pptx?alt=media: The billing account for the owning project is disabled in state closed: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)
An error occurred while processing 'data/2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx': 403 GET https://storage.googleapis.com/download/storage/v1/b/dauphine-bucket/o/data%2F2.1%20-%20Before%20Transformers%20-%20Gen%20AI%20-%20Dauphine%20Tunis.pptx?alt=media: The billing account for the owning project is disabled in state closed: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)
An error occurred while proc

### I.4. Visualizing the First Documents Extracted with LangChain

Explanation:

LangChain is a framework for developing applications powered by language models. We’ll use it to load and visualize the documents extracted from the file.

In [ ]:
for doc in documents[:3]:
    print(f"Content:\n{doc.page_content}\nMetadata:\n{doc.metadata}\n")

Content:
Generative AI with LLM
Metadata:
{'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-02T12:12:43', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '48f84e1bdca7ae95acb10169474c3135'}

Content:
Florian Bastin
Metadata:
{'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-02T12:12:43', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': 'f4361d35367e66b1a0e01ea107730df7'}

Content:
👨🏼‍🎓 Master MASH - Université PSL
Metadata:
{'source': './downloaded_files\\1 - G

### I.5. Join extracted document by page

Explanation:

- The text extraction block is uninformative because very small text blocks are extracted from the document.
- We can join the extracted text by page to get a more meaningful output.
- A metadata with the 'page_number' can be helpful
- The other metadatas need to be merged

In [ ]:
from collections import defaultdict

# Function to merge documents by page number
def merge_documents_by_page(documents: list[Document]) -> list[Document]:
    merged_documents: list[Document] = []
    page_dict = {}

    # Group documents by page number
    for doc in documents:
        page_number = doc.metadata.get('page_number')
        if page_number is not None:
            if page_number not in page_dict:
                page_dict[page_number] = [doc]
            else:
                page_dict[page_number].append(doc)

    # Merge documents for each page
    for page_number, docs in page_dict.items():
        if docs:
            # Use the metadata of the first document in the group
            merged_metadata = docs[0].metadata
            # Concatenate the page content of all documents in the group
            merged_content = "\n".join([doc.page_content for doc in docs])
            # Create a new Document with merged content and metadata
            merged_documents.append(Document(page_content=merged_content, metadata=merged_metadata))# TODO Add a document with merged content and metadata)

    return merged_documents

# Merge the documents by page
merged_documents = merge_documents_by_page(documents)

# Print the merged documents
for doc in merged_documents:
    print("-" * 50)
    print(f"Page Number: {doc.metadata.get('page_number')}")
    print(f"Content:\n{doc.page_content}\nMetadata:\n{doc.metadata}\n")
    print("-" * 50)


--------------------------------------------------
Page Number: 1
Content:
Generative AI with LLM
Florian Bastin
👨🏼‍🎓 Master MASH - Université PSL
👨🏼‍💻 LLM Engineer @OctoTechnology
Le Monde, Casino, Channel, Club Med, Pernod Ricard, Suez
‹#›
Generative AI with LLM
Florian Bastin
👨🏼‍🎓 Master MASH - Université PSL
👨🏼‍💻 LLM Engineer @OctoTechnology
Le Monde, Casino, Channel, Club Med, Pernod Ricard, Suez
‹#›
Generative AI with LLM
Florian Bastin
👨🏼‍🎓 Master MASH - Université PSL
👨🏼‍💻 LLM Engineer @OctoTechnology
Le Monde, Casino, Channel, Club Med, Pernod Ricard, Suez
‹#›
Generative AI with LLM
Florian Bastin
👨🏼‍🎓 Master MASH - Université PSL
👨🏼‍💻 LLM Engineer @OctoTechnology
Le Monde, Casino, Channel, Club Med, Pernod Ricard, Suez
‹#›
Metadata:
{'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-02T12:12:43', 'page_number': 1, 'languages': 

# II. Ingesting in Cloud SQL

We will ingest each merged_document in Cloud SQL.

ALREADY DONE by teacher: 
- Create a Cloud SQL instance
- Create a database in the instance


TODO:
- Create a table in CloudSQL with you initials
- Create the schema of the table
- Ingest the data in the table


Follow this [documentation](https://python.langchain.com/docs/integrations/vectorstores/google_cloud_sql_pg/)

### II.1 Understand how to connect to Cloud SQL 


First we need to connect to Cloud SQL 
- Follow this [link](https://cloud.google.com/sql/docs/postgres/connect-instance-auth-proxy) to understand how it works

Then be familiar ith the following PostgreSQL commands:
```bash 
`psql "host=127.0.0.1 port=5432 sslmode=disable dbname=gen_ai_db user=postgres"` # to connect to the user `postgres`
# the user we use is `students`
# a password provided by the teacher is required
`\l` # to list all databases
`\c gen_ai_db` # to connect to the database `gen_ai_db`
`\dt` # to list all tables
`\d+ table_name` # to describe a table
`SELECT * FROM table_name` # to select all rows from a table
`\du` # to list all users
`\q` # to quit
`CREATE DATABASE db_name;` # to create a database
`CREATE USER user_name WITH PASSWORD 'password';` # to create a user
`GRANT ALL PRIVILEGES ON DATABASE db_name TO user_name;` # to grant all privileges to a user on a database
`GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO user_name;` # to grant all privileges to a user on all tables in a schema
`ALTER USER user_name WITH SUPERUSER;` # to grant superuser privileges to a user
`DROP DATABASE db_name;` # to drop a database
`DROP USER user_name;` # to drop a user
`DROP TABLE table_name;` # to drop a table
`REVOKE ALL PRIVILEGES ON DATABASE db_name FROM user_name;` # to revoke all privileges from a user on a database
```

When Cloud SQL Proxy is downloaded and the tutorial is followed. You should be connected to the instance. 
You can connect to the dabase as a user `students` with the password provided by the teacher.
  - `psql "host=127.0.0.1 port=5432 sslmode=disable dbname=gen_ai_db user=students"`
  - Enter the password provided by the teacher
Try to create a table `initial_tests_table` with the following schema:
  - `CREATE TABLE initial_tests_table (id SERIAL PRIMARY KEY, document TEXT, page_number INT, title TEXT, author TEXT, date TEXT);`
  - `\dt` to check if the table has been created
  - `\d+ initial_tests_table` to check the schema of the table
  - `DROP TABLE initial_tests_table;` to drop the table
  - `\q` to quit


In [ ]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
from config import PROJECT_ID, REGION, INSTANCE, DATABASE, DB_USER
DB_PASSWORD = os.environ["DB_PASSWORD"]
TABLE_NAME = os.environ["TABLE_NAME"]

In [ ]:
from langchain_google_cloud_sql_pg import PostgresEngine

# Connect to the PostgreSQL database
engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE,
    region=REGION,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASSWORD,
)


In [ ]:
# Create a table in the PostgreSQL database with the required columns
from sqlalchemy.exc import ProgrammingError

try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME, # Vector size for VertexAI model(textembedding-gecko@latest)
        vector_size=768,
    )
except ProgrammingError:
    print("Table already created")

ClientResponseError: 409, message='Conflict', url='https://sqladmin.googleapis.com/sql/v1beta4/projects/dauphine-437611/instances/gen-ai-instance:generateEphemeralCert'

- Execute \d+ [YOUR_INITIALS]_table in the psql shell to check the schema of the table

### II.2 Create an embedding to convert your documents

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko",
    project=PROJECT_ID
)

In [ ]:
from langchain_google_cloud_sql_pg import PostgresVectorStore

vector_store = PostgresVectorStore.create_sync(  # Use .create() to initialize an async vector store
    engine=engine,
    table_name=TABLE_NAME,
    embedding_service=embedding,
)

In [ ]:
#vector_store.add_documents(merged_documents)
# Excute only once this cell

['efba080d-abdd-45ea-af9f-55c3e8901db0',
 '46d8a4b0-5249-4e51-ad2f-e91025b2cd6a',
 '05e96eb5-7061-427f-a876-5c20d21c7d88',
 '165a1178-972c-431c-9017-1186ff93f376',
 '7674057d-a9a1-4a0d-bcf7-0d08c99e30f6',
 '8ed2a30d-2fa0-4b77-baf0-308c52389475',
 'e2744621-8d73-4c7d-89f3-56d1fdcb820b',
 '3c95e7c5-97c7-45a9-9cbf-6a0627d24d09',
 'd3e80c19-33f8-473d-8c17-7dec639d4759',
 '27765abc-54a8-4fdb-bd49-8b9ec2120bde',
 '2a92dae3-3d16-4ecf-80b8-3c14747b8941',
 'c8a0c172-788b-4fb7-851e-5f90a9c89037',
 'b8d1bcfd-f50b-4b58-8e9a-d14e2743972c',
 '39398139-77f5-4cd5-a80e-7d57fde8b3a6',
 'f7b09388-e63d-4746-a7c5-316c16ae3136',
 'e58206e6-65a4-4865-8f77-89f7f1602953',
 'bc828649-d095-4206-ba7c-d9d910954a1f',
 '968ff0d3-afd5-4530-9462-46fdafa257bd',
 '68d9750a-0c75-4e9e-b84b-7074fae11f21',
 'c8f6dfb3-708f-47c7-bf4d-bbd6896d81a3',
 '7c385d6b-b608-4da6-aeeb-b7cb6beb0dfe',
 '650b427c-e567-4ca0-a864-1c09a00ce83e',
 '3713824d-47ac-4fef-b3b2-d4bc35370270',
 'd94a2ad4-a8a4-4530-8a4e-c0129baa22e6',
 'aaf375da-10f3-

In [ ]:
print(merged_documents)

[Document(metadata={'source': './downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': './downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-02T12:12:43', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '48f84e1bdca7ae95acb10169474c3135'}, page_content='Generative AI with LLM\nFlorian Bastin\n👨🏼\u200d🎓 Master MASH - Université PSL\n👨🏼\u200d💻 LLM Engineer @OctoTechnology\nLe Monde, Casino, Channel, Club Med, Pernod Ricard, Suez\n‹#›\nGenerative AI with LLM\nFlorian Bastin\n👨🏼\u200d🎓 Master MASH - Université PSL\n👨🏼\u200d💻 LLM Engineer @OctoTechnology\nLe Monde, Casino, Channel, Club Med, Pernod Ricard, Suez\n‹#›\nGenerative AI with LLM\nFlorian Bastin\n👨🏼\u200d🎓 Master MASH - Université PSL\n👨🏼\u200d💻 LLM Engineer @OctoTechnology\nLe Monde, Casino, Channel, Club Med, Pernod Ricard, Suez\n‹#›\nG

### II.3 Perform a similarity search

In [ ]:
query = "How to train a Large Language Model?"

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5, 
    }
)

docs = retriever.get_relevant_documents(query)

C:\Users\Youssef\AppData\Local\Temp\ipykernel_14876\1743924723.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [ ]:
for doc in docs:
    print("-" * 50)
    print("Content: ", doc.page_content)
    print("Metadata: ", doc.metadata)

--------------------------------------------------
Content:  I.A.7 Training Process
Training process
Steps 
Find scaling recipes (example: learning rate decrease if the size of the model increase)
Tune hyper parameters on small models of differents size
Choose the best models among the smallest ones
Train the biggest model with the 
Q. Should I use Transformers or LSTM ? 
‹#›
Stanford CS229 I Machine Learning I Building Large Language Models (LLMs) [Youtube]
II.A.3 RNN
Recurrent Neural Networks (Seq2seq model)
‹#›
II.B.1 Self Attention Mechanism
   are
Transformers
decoder
encoder
Query: What am I looking for ? 
|E| : Embedding (1, 12 288)
|WQ|: Query matrix (12 288, 128)
WQ
2.11
-4.22
..
..
5.93
2.43
-3.2
..
..
3.32
2.11
-4.22
..
..
1.12
3.11
-4.22
..
..
4.98
Embedding
3.23 -1.23 0.89 0.32 -3.29 3.23 1.23 -2.34 1.83 1.92 0.10 1.28
E2
E3
E1
2.11
-4.22
..
..
5.93
2.11
-4.22
..
..
5.93
2.11
-4.22
..
..
5.93
2.11
-4.22
..
..
5.93
Query		
Q1
Q2
Q3
Q4
Am I a superstar ?
Do I mean Allocation

**Congratulations**! You have successfully ingested the data in Cloud SQL.